In [1]:
# !pip install mediapipe opencv-python pandas scikit-learn

In [35]:
import mediapipe as mp
import cv2
import numpy as np  

In [36]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp.solutions.pose

In [37]:
VIDEO_PATH = 'videoData/deadlift/hips.avi'
EXPORT_PATH = 'annotations/deadlift/hips.csv'
MODEL_PATH = 'models/deadlift/hips.pkl'

1.1 Save Video

In [38]:
cap = cv2.VideoCapture(0)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter(VIDEO_PATH, cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))

cv2.VideoCapture()

while cap.isOpened():
    ret,frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    try:
        cv2.imshow('Press', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


2. Capture Landmars and Export to CSV

In [39]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [40]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [41]:
with open(EXPORT_PATH, mode='w', newline='') as f:
    csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [42]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist() #Converted to a list
        # print(keypoints)
        keypoints.insert(0,action)
        
        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [43]:
cap = cv2.VideoCapture(VIDEO_PATH) #name of the video
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        k = cv2.waitKey(1)
        if k== ord('a'):
            export_landmark(results,'wide')
        elif k== ord('s'):
            export_landmark(results,'neutral')
        elif k== ord('d'):
            export_landmark(results,'narrow')
        
        cv2.imshow('Hips Webcam Feed',image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Can't receive frame (stream end?). Exiting ...


### 3. Train Custom Model Using Scikit Learn

3.1 Read in Collected Data and Process

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [45]:
df = pd.read_csv(EXPORT_PATH)

In [46]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,narrow,0.600117,-0.060331,0.287223,0.999535,0.598248,-0.072766,0.238966,0.999300,0.594206,...,0.208524,0.902926,0.596525,0.922581,-0.012698,0.931774,0.619723,0.911863,0.127924,0.865272
1,narrow,0.594999,-0.053797,0.268718,0.999638,0.590773,-0.071037,0.220653,0.999525,0.584892,...,0.231640,0.936337,0.598702,0.919457,0.011906,0.918400,0.621777,0.909278,0.139202,0.865653
2,narrow,0.594326,-0.061186,0.267986,0.999652,0.590008,-0.076128,0.219101,0.999545,0.584259,...,0.232163,0.938389,0.599704,0.920391,0.018002,0.918446,0.622266,0.910872,0.142780,0.868487
3,narrow,0.593108,-0.066946,0.267649,0.999662,0.588626,-0.082501,0.218587,0.999561,0.583164,...,0.236602,0.940134,0.600252,0.920380,0.024109,0.917898,0.622406,0.911022,0.152518,0.870056
4,narrow,0.593265,-0.070031,0.239975,0.999657,0.589523,-0.084851,0.198420,0.999561,0.584443,...,0.231282,0.939716,0.601047,0.920385,0.019830,0.916628,0.622396,0.911401,0.144202,0.870111


In [47]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
851,narrow,0.582258,-0.022631,-0.544723,0.999214,0.594452,-0.056916,-0.535567,0.998242,0.602048,...,0.136911,0.841940,0.627146,0.941303,-0.051985,0.936994,0.515349,0.953488,-0.055363,0.924488
852,narrow,0.577241,-0.026807,-0.523650,0.999251,0.588541,-0.053377,-0.516261,0.998246,0.595557,...,0.139098,0.843581,0.628700,0.946187,-0.072592,0.941003,0.499683,0.955158,-0.066082,0.930031
853,narrow,0.573666,-0.032263,-0.438431,0.999251,0.584367,-0.055113,-0.432429,0.998094,0.591037,...,0.115911,0.844250,0.630481,0.947778,-0.060787,0.943374,0.484102,0.955960,-0.093074,0.934413
854,narrow,0.566907,-0.061731,-0.450452,0.999177,0.578957,-0.066538,-0.435944,0.997487,0.584849,...,0.093495,0.846809,0.629109,0.947053,0.005083,0.949321,0.479347,0.964366,-0.114690,0.945287
855,narrow,0.558643,-0.002342,-0.225337,0.999237,0.559811,-0.022292,-0.241031,0.997702,0.561103,...,0.091624,0.852663,0.503649,0.958727,-0.132555,0.950922,0.565497,0.947947,-0.059657,0.941761


In [48]:
df[df['class'] == 'up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33


In [49]:
X = df.drop('class', axis=1) #Features
Y = df['class'] #Target value

In [50]:
X_train, X_test, y_train,y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1234)

In [51]:
Y

0      narrow
1      narrow
2      narrow
3      narrow
4      narrow
        ...  
851    narrow
852    narrow
853    narrow
854    narrow
855    narrow
Name: class, Length: 856, dtype: object

3.2 Train Machine Learning Classification Model

In [52]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [53]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [54]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [55]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

3.3 Evaluate and Serialize Model


In [56]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle

In [57]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
            precision_score(y_test.values,yhat,average='weighted'),
            recall_score(y_test.values,yhat,average='weighted'))

lr 0.9922178988326849 0.9923151750972763 0.9922178988326849
rc 0.9961089494163424 0.9961334214325916 0.9961089494163424
rf 0.9961089494163424 0.9961334214325916 0.9961089494163424
gb 0.9922178988326849 0.9922423708489342 0.9922178988326849


In [58]:
with open(MODEL_PATH,'wb') as f:  #dumping the model
    pickle.dump(fit_models['rf'],f)

4. Make Detections with Model

In [59]:
with open(MODEL_PATH,'rb') as f:
    model = pickle.load(f)

In [60]:
cap = cv2.VideoCapture(0)
counter = 0


current_stage = ''
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detections
        results = pose.process(image)

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)

            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'up'
                counter += 1
                print(current_stage,counter)

            #Get status box
            cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

            #Display Class
            cv2.putText(image,'CLASS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Probability
            cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Counter
            cv2.putText(image,'COUNTER',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        except Exception as e:
            print(e)
            pass
        
        cv2.imshow('Raw Webcam Feed', image)

        if(cv2.waitKey(10) & 0xFF == ord('q')):
            break

cap.release()
cv2.destroyAllWindows()


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute